In [1]:
!pip install pysocks

In [8]:
df_car = pd.DataFrame(columns = ['페이지no','판매완료 여부','차량번호','차종','가격',
                                 '연식','주행거리','연료','변속기','연비','차종','배기량',
                                 '색상','세금미납','압류','저당','제시번호','사고내역여부',
                                 '전손이력','침수이력','용도이력','소유자변경이력','보험사고정보 조회일자',
                                 '옵션', '딜러이름','판매지역', '판매처','연락처','판매 중인 차량 수', '판매 완료 차량 수',
                                 '사용 기간', '연평균 주행거리',
                                 '내차 사고 피해 보험 횟수', 'KB 산출 시세 안전 구간', '출고가 대비 감가 비율'])

In [3]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import random

In [ ]:
# 셀레니움
from selenium import webdriver 
from selenium.webdriver import ActionChains 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC

# webdirver옵션에서 headless기능을 사용하겠다 라는 내용
webdriver_options = webdriver.ChromeOptions()
webdriver_options .add_argument('headless')


# setup Driver|Chrome : 크롬드라이버를 사용하는 driver 생성
# Chromedriver 이름 통으로 쓰면 블럭될 우려도 있어 파일명을 chro로 변경하여 사용
driver = webdriver.Chrome('C:\\Users\\WIN10\\PycharmProjects\\practice\\jupyter_src\\chromeee.exe', options=webdriver_options )
#driver = webdriver.Chrome('C:\\Users\\WIN10\\PycharmProjects\\practice\\jupyter_src\\chromeee.exe')


for number in range(22593516, 22700001):
    driver.implicitly_wait(3) # 암묵적으로 웹 자원을 (최대) 3초 기다리기

    url = 'https://www.kbchachacha.com/public/car/detail.kbc?carSeq='+str(number)
    
    driver.get(url)
    ## 글 제목 CSS Selector 이용하여 로딩되면 볼 수 있게 설정 // 시간은 너무 짧으면 봇 검사 페이지에 걸림. 10~15초 줌
    #element = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CSS_SELECTOR,"#content > div:nth-child(18) > div.common-sub-content.common-container.fix-content > div.cmm-cardt-area.adj1740.adj1670.adj1500.adj1441 > div.car-dt-info > div.car-buy-info > div.car-buy-price > strong")))
    
    element = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.CLASS_NAME,"car-buy-price")))
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
 
    # 리스승계 제외 (22383275)
    lease = soup.find('div',{'class':'suc-price'})

    #리스 승계 확인시 넘기기
    if lease != None:
        print(str(number) + ": 리스승계")
        continue
    
    #정보 리스트에 담기
    temp = []
    temp.append(number)
    
    print(number)
    
    sell_info = soup.find('div',{'class':'dim-txt'})
    if sell_info != None:
        temp.append("판매완료")
    else:
        temp.append("판매중")
    
    car_no = soup.find('strong',{'class':'car-buy-name'}).get_text().split(')')[0].replace("(","")
    ############## 차량번호 replace 걸 것
    car_name = soup.find('strong',{'class':'car-buy-name'}).get_text().replace("("+car_no+")","")
    price = soup.find('div',{'class':'car-buy-price'}).select('strong')[1].get_text().replace("만원", "")
    
    temp.append(car_no)
    temp.append(car_name)
    temp.append(price)
    
    # 차량정보
    detail1 = soup.find('div',{'class':'detail-info01'})
    detail2 = soup.find('div',{'class':'detail-info02'})
    detail3 = soup.find('div',{'class':'detail-info03'})

    # 기본정보(detail1)
    # 차량정보	143고3229	연식	19년11월(19년형)
    # 주행거리	55,268km	연료	가솔린
    # 변속기	오토	연비	정보없음
    # 차종	대형	배기량	5,038cc
    # 색상	회색	세금미납	없음
    # 압류	없음	저당	없음
    # 제시번호	20211211

    car_no = detail1.select('td')[0].get_text().strip()
    car_year = detail1.select('td')[1].get_text().strip()
    car_distance = detail1.select('td')[2].get_text().strip()
    car_fuel = detail1.select('td')[3].get_text().strip()
    car_gear = detail1.select('td')[4].get_text().strip()
    car_rate = detail1.select('td')[5].get_text().strip()
    car_size = detail1.select('td')[6].get_text().strip()
    car_cc = detail1.select('td')[7].get_text().strip()
    car_color = detail1.select('td')[8].get_text().strip()
    car_taxinfo = detail1.select('td')[9].get_text().strip()
    car_seize = detail1.select('td')[10].get_text().strip()
    car_security = detail1.select('td')[11].get_text().strip()
    car_secno = detail1.select('td')[12].get_text().strip()

    # 성능점검·보험사고이력정보 (detail2)
    # 성능점검기록부내역보기
    # 보험사고정보사고없음
    # 전손이력없음 / 침수이력없음 / 용도이력없음 / 소유자변경없음 / 보험사고정보 조회일자 : 2021.12.11

    accident_info = detail2.select('span')[3].get_text().strip()
    all_destroy = detail2.select('dd')[0].get_text().strip()
    flood = detail2.select('dd')[1].get_text().strip()
    usage = detail2.select('dd')[2].get_text().strip()
    user_change = detail2.select('dd')[3].get_text().strip()
    check_date = detail2.find('div',{'class':'mg-t40'}).select('span')[4].get_text()
    
    temp.append(car_year)
    temp.append(car_distance)
    temp.append(car_fuel)
    temp.append(car_gear)
    temp.append(car_rate)
    temp.append(car_size)
    temp.append(car_cc)
    temp.append(car_color)
    temp.append(car_taxinfo)
    temp.append(car_seize)
    temp.append(car_security)
    temp.append(car_secno)
    temp.append(accident_info)
    temp.append(all_destroy)
    temp.append(flood)
    temp.append(usage)
    temp.append(user_change)
    temp.append(check_date)
    
    #option은 리스트
    options = soup.find('ul',{'class':'car-option-list'}).select('li')
    option_text = ""

    ##전체보기 삭제
    for option in options:
        if option.get_text() == '등록된 주요 옵션이 없습니다.':
            break
        elif option.get_text() != '전체보기':
            option_text += option.get_text() + ","
  
    temp.append(option_text)
    
    # 딜러 정보
    dealer_info = soup.find('div',{'class':'dealer-cnt'})
    dealer_name = dealer_info.find('span', {'class':'name'}).get_text().replace("딜러", "")
    dealer_area = dealer_info.find('span', {'class':'place-add'}).get_text().split(" ")[0]
    dealer_place = dealer_info.find('span', {'class':'place-add'}).get_text()
    dealer_tel = soup.find('div',{'class':'dealer-tel-num'}).get_text()
    dealer_sellon = soup.find('span', {'id':'btnDealerHome3'})    
    
    #판매대수 관련 정보 없는 경우도 존재
    
    if dealer_sellon == None:
        dealer_sellon = ""
        dealer_sold = ""
    else:
        dealer_sellon = soup.find('span', {'id':'btnDealerHome3'}).get_text().split(":")[1].replace("대","")
        dealer_sold = soup.find('span', {'id':'btnDealerHome4'}).get_text().split(":")[1].replace("대","")

    temp.append(dealer_name)
    temp.append(dealer_area)
    temp.append(dealer_place)
    temp.append(dealer_tel)
    temp.append(dealer_sellon)
    temp.append(dealer_sold)
    
    # 주행거리(사용 기간 및 연평균 주행거리)
    distance_info = soup.find('div',{'class':'km-txt'})
    using_month = distance_info.find('span',{'class':'fs-16'}).get_text().replace("총 ","").replace(" 동안","")
    distance = distance_info.select('strong')[0].get_text()
    distance_per_year = distance_info.find('span',{'class':'mg-l5'}).get_text().split(" ")[1]
    
    temp.append(using_month)
    temp.append(distance_per_year)
    
    #내차 사고 피해 보험
    accident_info = soup.find('li',{'class':'used02'})
    accident = 0
    # accident = accident_info.find('span',{'class':'used-alarm'}).get_text()

    if accident_info.find('span',{'class':'used-alarm-none'}) == None:
         accident = accident_info.find('span',{'class':'used-alarm'}).get_text()
            
    temp.append(accident)
    
    # KB가 산출한 시세 안전 구간
    ai_price = soup.find('div',{'class':'cmm-price'}).select('strong')[0].get_text().replace("만원", "")

    temp.append(ai_price)
    
    #출고가 대비 감가 비율
    #차량의 최초 취득가격으로 선택옵션, 프로모션, 부가세가 포함된 실 구매가격입니다.

    reduction = soup.find('strong',{'class':'price-range-bar__current-number high modi'})
    
    if reduction != None:
        reduction = reduction.select('strong')[0].get_text().replace(" 감가", "")
    
    temp.append(reduction)
    
    print(temp)
    df_car = df_car.append(pd.Series(temp, index=df_car.columns), ignore_index=True)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24804\1603099136.py:15: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\\Users\\WIN10\\PycharmProjects\\practice\\jupyter_src\\chromeee.exe', options=webdriver_options )


22593516
[22593516, '판매중', '47거5198', '벤츠 GLK클래스GLK220 CDI4매틱 블루이피션시 X204', '1,470', '12년09월(13년형)', '200,000km', '디젤', '오토', '13.1Km', 'SUV', '2,143cc', '은색', '없음', '없음', '없음', '2021155207', '사고있음', '없음', '없음', '없음', '8회', '보험사고정보 조회일자 : 2022.02.07 ', '썬루프,내비게이션,오토라이트,주차감지센서,가죽시트,열선시트,사이드&커튼에어백,전동접이사이드미러,', '최지섭 ', '인천', '인천 엠파크타워자동차매매단지', '0503-5376-2903', '1', '42', '9년 2개월', '21,816Km', '4', '1,024~1,436', '73%']


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24804\1603099136.py:193: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_car = df_car.append(pd.Series(temp, index=df_car.columns), ignore_index=True)


22593517
[22593517, '판매완료', '69수4024', '현대 아반떼MDM16 GDI 프리미엄 ', '720', '11년06월(12년형)', '114,000km', '가솔린', '오토', '정보없음', '준중형', '1,600cc', '흰색', '없음', '없음', '없음', '2022020610', '사고있음', '없음', '없음', '없음', '4회', '보험사고정보 조회일자 : 2022.02.07 ', '', '남기현 ', '경기', '경기 DY카랜드', '0507-0472-3364', '14', '215', '10년 6개월', '10,848Km', '1', '448~627', '61%']


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24804\1603099136.py:193: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_car = df_car.append(pd.Series(temp, index=df_car.columns), ignore_index=True)


...

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24804\1603099136.py:193: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_car = df_car.append(pd.Series(temp, index=df_car.columns), ignore_index=True)


22595054
[22595054, '판매완료', '39어8011', '현대 아반떼HD1.6 VVT S16 LUXURY', '420', '07년03월(07년형)', '67,628km', '가솔린', '오토', '정보없음', '준중형', '0cc', '은색', '정보없음', '없음', '없음', '20221026673', '사고없음', '없음', '없음', '없음', '3회', '보험사고정보 조회일자 : 2022.02.07 ', '내비게이션,오토라이트,열선시트,전동접이사이드미러,알루미늄휠,', '박동진 ', '경기', '경기 SKV1모터스매매단지', '0504-860-7008', '7', '39', '14년 8개월', '4,608Km', 0, '276~390', '71%']


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24804\1603099136.py:193: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_car = df_car.append(pd.Series(temp, index=df_car.columns), ignore_index=True)


22595055
[22595055, '판매완료', '38누3528', '폭스바겐 뉴 CC2.0 TDI 블루모션 4MOTION ', '1,000', '13년10월(14년형)', '190,582km', '디젤', '오토', '정보없음', '중형', '1,968cc', '흰색', '없음', '없음', '없음', '20211347300', '사고있음', '없음', '없음', '없음', '1회', '보험사고정보 조회일자 : 2022.02.07 ', '썬루프,내비게이션,스마트키,열선시트,크루즈컨트롤,', '양회찬 ', '경기', '경기 수원오토컬렉션', '0504-882-9974', '86', '1,553', '8년 2개월', '23,328Km', '2', '919~1,276', '79%']


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24804\1603099136.py:193: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_car = df_car.append(pd.Series(temp, index=df_car.columns), ignore_index=True)


In [22]:
df_car

,페이지no,판매완료 여부,차량번호,차종,가격,연식,주행거리,연료,변속기,연비,...,판매지역,판매처,연락처,판매 중인 차량 수,판매 완료 차량 수,사용 기간,연평균 주행거리,내차 사고 피해 보험 횟수,KB 산출 시세 안전 구간,출고가 대비 감가 비율
0,22551350,판매완료,169저2073,기타 현대(상용)굴삭기2900,"9,999",(년형),1km,,,정보없음,...,경기,경기 도이치오토월드,,"2,236","25,165",1개월,12Km,0,"8,850~15,224",None
1,22551352,판매중,84루6385,현대 포터2초장축 더블캡 CRDi Smart,"1,650",18년05월(18년형),"72,853km",디젤,오토,정보없음,...,경기,경기 남수원자동차매매단지,0507-0474-5040,28,187,3년 8개월,"19,860Km",0,"1,829~2,096",12%
2,22551353,판매완료,175가1829,기타 현대(상용)굴삭기2900,"9,999",(년형),1km,,,정보없음,...,경기,경기 도이치오토월드,,"2,236","25,165",1개월,12Km,3,"8,915~14,962",None
3,22551355,판매완료,53더5565,기아 레이1.0 가솔린 럭셔리,760,12년10월(13년형),"52,405km",가솔린,오토,정보없음,...,경기,경기 SKV1모터스매매단지,,5,108,9년 1개월,"5,760Km",0,529~738,47%
4,22551356,판매완료,18소0673,기아 K5가솔린 2.0 프레스티지,630,11년11월(12년형),"177,290km",가솔린,오토,정보없음,...,인천,인천 인천교자동차매매단지,0507-2018-6085,9,129,10년 1개월,"17,580Km",3,444~620,78%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34838,22593508,판매중,12구4501,기아 쏘렌토R디젤 2.0 2WD TLX 최고급형,919,10년01월(10년형),"102,698km",디젤,오토,정보없음,...,인천,인천 엠파크허브자동차매매단지,0503-5377-1249,13,127,11년 10개월,"8,676Km",1,"862~1,079",67%
34839,22593511,판매중,63모8066,기아 더 프레스티지 K73.0 GDI 프레스티지,530,11년03월(12년형),"190,538km",가솔린,오토,정보없음,...,인천,인천 엠파크타워자동차매매단지,0504-859-0796,7,7,10년 8개월,"17,856Km",2,482~675,86%
34840,22593512,판매완료,40구5612,BMW 올뉴5시리즈 (G30)530d M Sport Plus,"10,029",17년10월(17년형),1km,디젤,오토,13.5Km,...,경기,경기 도이치오토월드,0503-5389-7862,"2,236","25,165",4년 3개월,0Km,1,"4,222~5,870",None
34841,22593513,판매완료,58어6666,벤츠 뉴 S클래스S600L W222(13~),"6,430",14년06월(15년형),"65,957km",가솔린,오토,정보없음,...,경기,경기 수원오토컬렉션,0504-883-1280,56,92,7년 7개월,"8,688Km",3,"3,622~5,528",76%


In [23]:
df_car.to_csv('중고차 크롤링자료_22560000~22593514.csv', encoding='utf8')